In [1]:
from langgraph.graph import END, START, StateGraph
from langchain_ollama import ChatOllama
from typing import TypedDict , Annotated
from pydantic import BaseModel,Field
import operator

In [2]:
# 1. Init Local Model
model = ChatOllama(
    model="llama3.2:3b",
    temperature=0.7
)

schema for o/p

In [3]:
# 2. Schema
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="Score for the essay out of 10", ge=0, le=10)


structured_model = model.with_structured_output(EvaluationSchema)

essay

In [4]:
essay = """
Artificial Intelligence (AI) has emerged as one of the most transformative forces shaping modern India. With rapid advancements in machine learning, natural language processing, robotics, and data analytics, AI is redefining industries, improving governance, and enabling new forms of economic growth. As a nation with a vast population, diverse economy, and strong technological talent, India is uniquely positioned to harness AI for inclusive development.

One of the most significant roles of AI in India lies in economic expansion and innovation. Sectors such as IT services, finance, manufacturing, and retail are increasingly adopting AI-driven solutions to enhance productivity and reduce operational costs. India’s tech ecosystem—anchored by companies like TCS, Infosys, Wipro and a growing community of startups—has rapidly integrated AI into automation, predictive analytics, and intelligent business operations. This has strengthened India’s position as a global technology hub.

In governance and public administration, AI plays a critical role through initiatives under Digital India and India Stack. AI-powered platforms support digital payments, Aadhaar-based services, and citizen grievance handling. State governments use machine learning models for early disaster warning systems, traffic management, land record digitization, and crop advisory services. These innovations improve transparency, reduce corruption, and make government services more efficient and accessible.

AI also has transformative potential in healthcare, where India faces challenges of doctor shortages and uneven access. AI-enabled diagnostics, telemedicine, image analysis, and predictive health monitoring are helping bridge gaps in rural and underserved regions. Startups and research labs are developing AI tools for early detection of diseases such as tuberculosis, cancer, and diabetic retinopathy, improving public health outcomes.

Another major domain is agriculture, where AI supports precision farming, weather forecasting, soil analysis, crop disease detection, and supply chain optimization. These tools help farmers make informed decisions, reduce losses, and increase productivity. Government collaborations with companies like Microsoft and IBM have already shown success in providing AI-based advisory services to thousands of farmers.

In education, AI-driven platforms support personalized learning, skill development, and language translation, especially beneficial in a multi-lingual country like India. The National Education Policy (2020) highlights the use of AI for adaptive learning and teacher training, making learning more accessible and tailored to individual student needs.

Despite its immense potential, AI adoption in India faces challenges such as lack of skilled workforce, data privacy concerns, limited research infrastructure, and uneven deployment across states. To address these gaps, the Government of India launched the National Strategy for Artificial Intelligence (AI for All), focusing on responsible AI, innovation, and building a strong AI ecosystem. Investments in semiconductor manufacturing, AI research hubs, and startup incubation are accelerating the nation’s progress toward becoming a global AI leader.

In conclusion, AI holds a pivotal role in India’s journey toward economic growth, social inclusion, and technological leadership. With strategic investments, ethical governance, and continuous innovation, AI has the power to reshape India into a digitally empowered nation where technology benefits every citizen. The coming years will be crucial as India balances AI-driven advancement with equity, responsibility, and long-term sustainability.
"""

In [5]:
# 4. Prompt (use f-string!)
prompt = f"""
Evaluate the language quality of the following essay.
Provide detailed feedback and assign a score out of 10.

Essay:
{essay}
"""

In [6]:
# 5. Invoke
result = structured_model.invoke(prompt)
print(result)

feedback="The essay provides an overview of the transformative potential of Artificial Intelligence (AI) in modern India, highlighting its applications in various sectors such as economy, governance, healthcare, agriculture, education, and technology. The writer successfully conveys the significance of AI in driving economic growth, improving governance, enhancing public services, and promoting social inclusion. The essay provides specific examples of AI-driven innovations and initiatives across different domains, showcasing India's unique position to harness AI for inclusive development. However, there are areas where the writing could be more engaging, and some sections lack depth or clarity. Overall, the score is 8 out of 10." score=8


In [7]:
class UPSCState(TypedDict):
    essay: str                                  # Read-only
    language_feedback: str                      # written by ONLY 1 node
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]  # parallel-safe
    avg_score: float

#nodes


In [8]:
def evaluate_language(state: UPSCState) -> UPSCState:
    prompt = f"""
    Evaluate LANGUAGE quality. Give score out of 10.
    Essay:
    {state["essay"]}
    """
    out = structured_model.invoke(prompt)

    return {
        "language_feedback": out.feedback,
        "individual_scores": [out.score]   # IMPORTANT: a list
    }


In [9]:
def evaluate_analysis(state: UPSCState) -> UPSCState:
    prompt = f"""
    Evaluate DEPTH OF ANALYSIS. Give score out of 10.
    Essay:
    {state["essay"]}
    """
    out = structured_model.invoke(prompt)

    return {
        "analysis_feedback": out.feedback,
        "individual_scores": [out.score]
    }

In [10]:
def evaluate_clarity(state: UPSCState) -> UPSCState:
    prompt = f"""
    Evaluate CLARITY OF THOUGHT. Give score out of 10.
    Essay:
    {state["essay"]}
    """
    out = structured_model.invoke(prompt)

    return {
        "clarity_feedback": out.feedback,
        "individual_scores": [out.score]
    }

In [11]:
def evaluate_clarity(state: UPSCState) -> UPSCState:
    prompt = f"""
    Evaluate the CLARITY OF THOUGHT of the following essay.
    Give detailed feedback and a score out of 10.

    Essay:
    {state["essay"]}
    """
    out = structured_model.invoke(prompt)
    return {
        **state,
        "clarity_feedback": out.feedback,
        "individual_scores": state["individual_scores"] + [out.score]
    }

In [12]:
def evaluate_overall(state: UPSCState) -> UPSCState:
    summary_prompt = f"""
    Combine these into final UPSC-style evaluation:

    LANGUAGE:
    {state["language_feedback"]}

    ANALYSIS:
    {state["analysis_feedback"]}

    CLARITY:
    {state["clarity_feedback"]}
    """

    overall = model.invoke(summary_prompt).content
    avg_score = sum(state["individual_scores"]) / len(state["individual_scores"])

    return {
        "overall_feedback": overall,
        "avg_score": avg_score
    }

#graph

In [13]:
graph = StateGraph(UPSCState)

graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_clarity", evaluate_clarity)
graph.add_node("evaluate_overall", evaluate_overall)

# Parallel fan-out
graph.add_edge(START, "evaluate_language")
graph.add_edge(START, "evaluate_analysis")
graph.add_edge(START, "evaluate_clarity")

# Merge all into final node
graph.add_edge("evaluate_language", "evaluate_overall")
graph.add_edge("evaluate_analysis", "evaluate_overall")
graph.add_edge("evaluate_clarity", "evaluate_overall")

graph.add_edge("evaluate_overall", END)

workflow = graph.compile()


In [14]:
essay1 = """
Artificial Intelligence (AI) has emerged as one of the most transformative forces shaping modern India. With rapid advancements in machine learning, natural language processing, robotics, and data analytics, AI is redefining industries, improving governance, and enabling new forms of economic growth. As a nation with a vast population, diverse economy, and strong technological talent, India is uniquely positioned to harness AI for inclusive development.

One of the most significant roles of AI in India lies in economic expansion and innovation. Sectors such as IT services, finance, manufacturing, and retail are increasingly adopting AI-driven solutions to enhance productivity and reduce operational costs. India’s tech ecosystem—anchored by companies like TCS, Infosys, Wipro and a growing community of startups—has rapidly integrated AI into automation, predictive analytics, and intelligent business operations. This has strengthened India’s position as a global technology hub.

In governance and public administration, AI plays a critical role through initiatives under Digital India and India Stack. AI-powered platforms support digital payments, Aadhaar-based services, and citizen grievance handling. State governments use machine learning models for early disaster warning systems, traffic management, land record digitization, and crop advisory services. These innovations improve transparency, reduce corruption, and make government services more efficient and accessible.

AI also has transformative potential in healthcare, where India faces challenges of doctor shortages and uneven access. AI-enabled diagnostics, telemedicine, image analysis, and predictive health monitoring are helping bridge gaps in rural and underserved regions. Startups and research labs are developing AI tools for early detection of diseases such as tuberculosis, cancer, and diabetic retinopathy, improving public health outcomes.

Another major domain is agriculture, where AI supports precision farming, weather forecasting, soil analysis, crop disease detection, and supply chain optimization. These tools help farmers make informed decisions, reduce losses, and increase productivity. Government collaborations with companies like Microsoft and IBM have already shown success in providing AI-based advisory services to thousands of farmers.

In education, AI-driven platforms support personalized learning, skill development, and language translation, especially beneficial in a multi-lingual country like India. The National Education Policy (2020) highlights the use of AI for adaptive learning and teacher training, making learning more accessible and tailored to individual student needs.

Despite its immense potential, AI adoption in India faces challenges such as lack of skilled workforce, data privacy concerns, limited research infrastructure, and uneven deployment across states. To address these gaps, the Government of India launched the National Strategy for Artificial Intelligence (AI for All), focusing on responsible AI, innovation, and building a strong AI ecosystem. Investments in semiconductor manufacturing, AI research hubs, and startup incubation are accelerating the nation’s progress toward becoming a global AI leader.

In conclusion, AI holds a pivotal role in India’s journey toward economic growth, social inclusion, and technological leadership. With strategic investments, ethical governance, and continuous innovation, AI has the power to reshape India into a digitally empowered nation where technology benefits every citizen. The coming years will be crucial as India balances AI-driven advancement with equity, responsibility, and long-term sustainability.
"""

In [15]:
initial_state = {
    "essay": essay,
}

In [16]:
result = workflow.invoke(initial_state)
print(result)

{'essay': '\nArtificial Intelligence (AI) has emerged as one of the most transformative forces shaping modern India. With rapid advancements in machine learning, natural language processing, robotics, and data analytics, AI is redefining industries, improving governance, and enabling new forms of economic growth. As a nation with a vast population, diverse economy, and strong technological talent, India is uniquely positioned to harness AI for inclusive development.\n\nOne of the most significant roles of AI in India lies in economic expansion and innovation. Sectors such as IT services, finance, manufacturing, and retail are increasingly adopting AI-driven solutions to enhance productivity and reduce operational costs. India’s tech ecosystem—anchored by companies like TCS, Infosys, Wipro and a growing community of startups—has rapidly integrated AI into automation, predictive analytics, and intelligent business operations. This has strengthened India’s position as a global technology 